# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-2584000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# both vulnerable. you are sitting North as dealer and you hold
hand = '5.A97643.Q82.QJ5'

# the auction goes:
auction = ["PAD_START", "PAD_START", "PAD_START", "1C", "PASS", "2S", "PASS", "3S", "PASS", "4D",  "PASS", "4S", "PASS", "4N", "PASS", "5D", "PASS", "6S", "PASS", "PASS", "PASS"]

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

Loading DDS: ../..\bin\dds.dll


Loaded lib dds.dll


In [4]:
lead.card

HA

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [5]:
lead.to_dict()['candidates']

[{'card': 'HA',
  'insta_score': 0.275,
  'expected_tricks_sd': 11.3,
  'p_make_contract': 0.5},
 {'card': 'Dx',
  'insta_score': 0.159,
  'expected_tricks_sd': 11.6,
  'p_make_contract': 0.5},
 {'card': 'CQ',
  'insta_score': 0.509,
  'expected_tricks_sd': 11.9,
  'p_make_contract': 0.4}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [6]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['98xx.Kx.A9.KT8xx ATx.J8x.xxx.9xxx KQJxx.QT.KJTxx.A 0.28185',
 'Txx.Tx.AK.K9xxxx KJxx.Jx.9xxx.T8x AQ98x.KQ8.JTxx.A 0.25795',
 'T98x.8x.Kx.AKTxx Jxx.QTx.9xx.98xx AKQxx.KJ.AJTxx.x 0.25369',
 'Jxx.8x.AK.KT9xxx 98xx.QJTx.JTx.8x AKQTx.K.9xxxx.Ax 0.25150',
 'Kxx.KQx.xx.KT9xx ATxx.T8x.T9xx.xx QJ98x.J.AKJx.A8x 0.25003',
 'K9xx.KJ.Jx.Kxxxx T8x.8xx.xxxx.ATx AQJxx.QT.AKT9.98 0.24588',
 'AJxx.QJx.x.KT9xx T9x.T8.KJxxx.8xx KQ8xx.Kx.AT9x.Ax 0.24342',
 'J9x.KQx.Kxx.KT9x KTxx.T8.9xx.8xxx AQ8xx.Jx.AJTx.Ax 0.24322',
 'A9xx.KJx.xx.KT9x T8x.T8x.KT9x.8xx KQJxx.Q.AJxx.Axx 0.24222',
 'Jxx.KJTx.KJ.KT8x KT98.8x.Txxx.xxx AQxxx.Q.A9xx.A9x 0.24202']